<a href="https://colab.research.google.com/github/unjuken/scalene_svm/blob/main/scalene_svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>